# header file

In [1]:
import numpy as np
import pandas as pd
print('done')

done


In [2]:
np.set_printoptions(suppress=True) #prevent numpy exponential 

In [3]:
path = "/media/mint/New Volume/4_semester_IIIT_D/capstone/hmog_dataset/180679/180679_session_"#13/TouchEvent.csv'

In [4]:
# screen grid (splitting the user screen into y_cross * x_cross)
y_cross = 8
x_cross = 6

# getting the screen size

In [5]:
# to get the minimum and maximum dimension of user screen size 

def screen_size():
    xx_min = 100000
    xx_max = -1
    yy_min = 100000
    yy_max = -1
    for i in range(1,25):
        data = pd.read_csv(path+str(i)+"/TouchEvent.csv", header=None)
        # data.head()
        x_min = np.min(data.iloc[:,6])# 6 and 7 th are the x and y axis of csv file
        x_max = np.max(data.iloc[:,6])
        y_min = np.min(data.iloc[:,7])
        y_max = np.max(data.iloc[:,7])
        if(x_min < xx_min):
            xx_min = x_min
        if(y_min < yy_min):
            yy_min = y_min
        if(x_max > xx_max):
            xx_max = x_max
        if(y_max > yy_max):
            yy_max = y_max
    # print(xx_min)
    # print(xx_max)
    # print(yy_min)
    # print(yy_max)

    x_len = xx_max - xx_min
    # print(x_len)
    fact = x_len / x_cross 
    # print(fact)
    x_axis = np.linspace(fact, x_len-fact, x_cross-1)
    print(x_axis)

    y_len = yy_max - yy_min
    # print(y_len)
    fact = y_len / y_cross 
    # print(fact)
    y_axis = np.linspace(fact, y_len-fact, y_cross-1)
    print(y_axis)
    return x_axis, y_axis

In [6]:
x_axis, y_axis = screen_size()

[ 174.75  349.5   524.25  699.    873.75]
[  259.38265625   518.7653125    778.14796875  1037.530625    1296.91328125
  1556.2959375   1815.67859375]


# POLICY iteration: parameter initialisation

In [7]:

NO_OF_STATES = 48#24#36#80#48#
NO_OF_ACTIONS = 2# 0 means remain in same state, 1 otherwise  
TRANSITION_PROBABILITY= np.zeros((NO_OF_STATES, NO_OF_ACTIONS, NO_OF_STATES))  # transition probability
TRANSITION_PROBABILITY= TRANSITION_PROBABILITY+ 0.00001
print(TRANSITION_PROBABILITY.shape)
REWARD = np.zeros((NO_OF_STATES, NO_OF_ACTIONS, NO_OF_STATES))  # rewards
REWARD =REWARD + 0.00001 # - 10.1
print(REWARD.shape)

POLICY = [0 for s in range(NO_OF_STATES)]
V = np.zeros(NO_OF_STATES)

GAMMA = 0.7

(48, 2, 48)
(48, 2, 48)


# POLICY iteration

In [8]:
def POLICY_iteration():
    flag = True # whether value changed or not

    while flag:
        flag = False

        for s in range(NO_OF_STATES):
            temp = 0
            for s1 in range(NO_OF_STATES):
                temp += (REWARD[s,POLICY[s],s1] + GAMMA*V[s1]) * TRANSITION_PROBABILITY[s,POLICY[s],s1]   
            V[s] = temp
                
        for s in range(NO_OF_STATES):
            cur = V[s]

            for a in range(NO_OF_ACTIONS):
                temp = 0
                for s1 in range(NO_OF_STATES):
                    temp += (REWARD[s, a, s1] + GAMMA * V[s1]) * TRANSITION_PROBABILITY[s, a, s1] 
                
                if temp > cur:
                    flag = True
                    POLICY[s] = a
                    cur = temp


    print "POLICY:  "
    policy_temp = np.zeros((y_cross,x_cross))
    v_temp = np.zeros((y_cross,x_cross))
    for i in range(y_cross):
        for j in range(x_cross):
            #print(i,j), 
#             print(i*x_cross+j),
            policy_temp[i,j] = POLICY[i*x_cross+j]
            v_temp[i,j] = V[i*x_cross+j]

    print policy_temp
    print v_temp

# Initially data is in milisecond
    
    so milisecond must be combined to form some meaningful things
    
# So, Milisecond data is combined into .1 seconds

In [9]:
def user_data_generation(start_user_id, end_user_id):
    user_movement_data = []

    count = 0
    for k in range(start_user_id, end_user_id+1):
        data = pd.read_csv(path+str(k)+"/TouchEvent.csv", header=None)
        #count = 0
        i = 0
        #for i in range(len(data)-1):
        while i < len(data): 
            #count += 1# total number of rows
            x_cor = []
            y_cor = []
            total_pressure = 0

            time1 = data.iloc[i,1]

            j = i
            while j < len(data):

                time2 = data.iloc[j,1]

                if time2-time1<=100: # .1 seconds session is combined
                    x = data.iloc[j,6]
                    y = data.iloc[j,7]
                    pressure = data.iloc[j,8]

                    x_cor.append(x)
                    y_cor.append(y)
                    total_pressure += pressure    

                    j+= 1

                else:
                    break

            x = np.mean(x_cor)
            y = np.mean(y_cor)

            user_movement_data.append([x,y,total_pressure])

            count = count +1
            i = j

    print(count)

    user_movement_data = pd.DataFrame(user_movement_data)

    return user_movement_data
                 

# Initial 20 session for training

# Last 4 session for testing

In [10]:
user_movement_data = user_data_generation(1,20)
print(user_movement_data.head())
user_movement_data_test = user_data_generation(21,24)
print(user_movement_data_test.head())

11660
            0     1    2
0  165.033126  33.0  5.0
1  146.992019  33.0  7.0
2  147.000000  93.0  5.0
3  144.566273  93.0  6.0
4  679.000000  64.0  6.0
2203
            0           1     2
0  467.454545  959.545455  11.0
1  444.153846  908.846154  13.0
2  444.534991  931.909600   8.0
3  450.237385  962.535321  14.0
4  454.410353  970.843275  12.0


# POLICY iteration is applied on data after 1 second time interval

In [11]:
alpha = 0.5
# for j in range(1,2):
#data = pd.read_csv(path+str(j)+"/TouchEvent.csv", header=None)
count = 0
for i in range(len(user_movement_data)-1):
    x = np.digitize(x = [user_movement_data.iloc[i,0]], bins = x_axis)[0]
    y = np.digitize(x = [user_movement_data.iloc[i,1]], bins = y_axis)[0]

    x_next = np.digitize(x = [user_movement_data.iloc[i+1,0]], bins = x_axis)[0]
    y_next = np.digitize(x = [user_movement_data.iloc[i+1,1]], bins = y_axis)[0]

    pressure = user_movement_data.iloc[i,2]
    #contact_size = data.iloc[i,9]

    state1 = y*x_cross+x
    state2 = y_next * x_cross+ x_next

    if state1 == state2:
        action = 0
    else:
        action = 1

    TRANSITION_PROBABILITY[state1][action][state2] += 1#0.001#1
    REWARD[state1][action][state2] += 1 + alpha * pressure#0.001 * pressure#(pressure * contact_size)

    count = count + 1
    if count % 600 == 0: #  # 50 times for 5 seconds, 600 times for 1 minutes(60 seconds)

        for k in range(NO_OF_STATES):
            TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
            TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

        POLICY_iteration()


# POLICY_iteration()
for k in range(NO_OF_STATES):
    TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
    TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

# POLICY_iteration()

print(count)
print('done')

POLICY:  
[[ 1.  0.  1.  1.  1.  1.]
 [ 1.  0.  1.  1.  1.  1.]
 [ 0.  1.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  1.  1.]
 [ 0.  1.  1.  0.  0.  1.]
 [ 0.  1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[  222.43553049   237.61600853    78.46413628   107.69671911
     77.96437117   154.70144721]
 [  215.94514746   237.61600853   289.43779501   203.37464204
    275.47139254   158.24300538]
 [  237.61600853   384.53561651   621.64572652   407.08944923
    384.48160536   287.60622522]
 [  237.61600853   389.98504018   949.97246576  1169.97157047
    521.73786197   404.47166503]
 [  237.61600853   618.41057203   587.1767659    788.3147872    456.65434662
    411.77605212]
 [  237.61600853   247.11577679   428.1153516    402.82316629
    420.79021875   350.17186669]
 [  237.61600853   237.61600853   237.61600853   237.61600853
    237.61600853   237.61600853]
 [  237.61600853   237.61600853   237.61600853   237.61600853
    237.61600853   237.61600853]]
POLICY:  
[[

# testing phase

In [12]:
# error = 0.0
correct_count = 0
acc = []


time_count = 0
for i in range(len(user_movement_data_test)-1):
    x = np.digitize(x = [user_movement_data_test.iloc[i,0]], bins = x_axis)[0]
    y = np.digitize(x = [user_movement_data_test.iloc[i,1]], bins = y_axis)[0]

    x_next = np.digitize(x = [user_movement_data_test.iloc[i+1,0]], bins = x_axis)[0]
    y_next = np.digitize(x = [user_movement_data_test.iloc[i+1,1]], bins = y_axis)[0]

    pressure = user_movement_data_test.iloc[i,2]
    #contact_size = data.iloc[i,9]

    state1 = y*x_cross+x
    state2 = y_next * x_cross+ x_next

    if state1 == state2:
        action = 0
    else:
        action = 1
    
    # checking for whether user have taken right action or not
    if POLICY[state1] == action:
        correct_count += 1
        
    #comment
    TRANSITION_PROBABILITY[state1][action][state2] += 1#0.001#1
    REWARD[state1][action][state2] += 1 + 0.5 * pressure#0.001 * pressure#(pressure * contact_size)

    time_count = time_count + 1
    if time_count % 300 == 0: #  # 50 times for 5 seconds, 600 times for 1 minutes(60 seconds)

        acc_in = (100*(correct_count))/300.0
        acc.append(acc_in)
        
        correct_count = 0

        # comment
        for k in range(NO_OF_STATES):
            TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
            TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

        POLICY_iteration()


# comment
for k in range(NO_OF_STATES):
    TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
    TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

POLICY_iteration()


print(time_count)


acc_in = (100.0 *(correct_count))/ (time_count%300)
acc.append(acc_in)


POLICY:  
[[ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  0.  0.  1.  1.]
 [ 0.  1.  1.  0.  0.  1.]
 [ 1.  1.  1.  0.  0.  1.]
 [ 1.  1.  1.  0.  1.  1.]
 [ 0.  1.  1.  1.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.]]
[[  3372.41254591   3301.16337482   7081.42886645   4281.34719455
    3014.37231539   3283.56968448]
 [  3809.59341192   3934.74202566   5561.44451198   5855.22344403
    4034.86715922   4212.03414167]
 [  3100.257855     4084.29112966   6015.00003333   6755.00003333
    7150.59755146   7665.02321821]
 [  3530.48699415   4517.17959685   6071.74201125  11346.6667
    8758.33336667   5075.06651803]
 [  2818.12999807   4678.03556295   9119.39222636  15250.00003333
    7903.33336667   4952.71201649]
 [  2108.14415844   2456.81465864   4732.23018613   7236.6667
    5128.37165654   3613.52537659]
 [  3530.48699415   2999.78757481   4434.47964166   4767.00017084
    4917.38241544   3530.48699415]
 [  3530.48699415   3530.48699415   3530.48699415   3530.48699415
    4449

In [13]:
# with modification
print(acc)
print(np.sum(acc)/len(acc))

[54.666666666666664, 55.0, 56.0, 61.666666666666664, 59.333333333333336, 32.0, 23.666666666666668, 12.745098039215685]
44.3848039216


In [14]:
# [54.666666666666664, 55.0, 56.0, 61.666666666666664, 59.333333333333336, 32.0, 23.666666666666668, 12.745098039215685]
# 44.3848039216